# 4 Routing and directions

https://azure.microsoft.com/en-us/products/azure-maps/?msockid=2e39c66c693c66a5151fd200687567d0

https://learn.microsoft.com/en-us/azure/azure-maps/

In [1]:
import folium
import pandas as pd
import os
import polyline
import requests
import sys

from datetime import datetime, timedelta
from dotenv import load_dotenv
from IPython.display import IFrame, FileLink
from typing import List, Dict, Tuple, Optional

## Settings

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
load_dotenv("azure.env")

True

In [4]:
RESULTS_DIR = "results"

os.makedirs(RESULTS_DIR, exist_ok=True)

In [5]:
class AzureMapsClient:
    """
    Azure Maps API Client for Python
    """
    def __init__(self, subscription_key: str):
        """
        Initialize Azure Maps client
        
        Args:
            subscription_key: Your Azure Maps subscription key
        """
        self.subscription_key = subscription_key
        self.base_url = "https://atlas.microsoft.com"

    def _make_request(self, endpoint: str, params: Dict) -> Dict:
        """
        Make authenticated request to Azure Maps API
        
        Args:
            endpoint: API endpoint
            params: Request parameters
            
        Returns:
            API response as dictionary
        """
        params['api-version'] = '1.0'
        params['subscription-key'] = self.subscription_key

        url = f"{self.base_url}/{endpoint}"

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return {}

In [6]:
azure_maps = AzureMapsClient(os.getenv('AZURE_MAPS_KEY'))

## Helper

In [7]:
class RoutingDemo:
    """
    Demonstrates Azure Maps routing capabilities
    """
    def __init__(self, azure_maps_client):
        self.client = azure_maps_client

    def calculate_route(self,
                        waypoints: List[Tuple[float, float]],
                        travel_mode: str = "car",
                        route_type: str = "fastest",
                        avoid: List[str] = None,
                        departure_time: str = None) -> Dict:
        """
        Calculate route between waypoints
        
        Args:
            waypoints: List of (latitude, longitude) tuples
            travel_mode: Transportation mode (car, truck, taxi, bus, van, motorcycle, bicycle, pedestrian)
            route_type: Route type (fastest, shortest, eco, thrilling)
            avoid: List of road types to avoid (tollRoads, motorways, ferries, unpavedRoads, carpools)
            departure_time: Departure time in ISO format
            
        Returns:
            Route calculation results
        """
        # Format waypoints as lat,lon pairs separated by colons
        query_points = ":".join([f"{lat},{lon}" for lat, lon in waypoints])

        params = {
            'query': query_points,
            'travelMode': travel_mode,
            'routeType': route_type,
            'computeTravelTimeFor': 'all',
            'sectionType': 'traffic'
        }

        if avoid:
            params['avoid'] = ",".join(avoid)

        if departure_time:
            params['departAt'] = departure_time

        response = self.client._make_request('route/directions/json', params)
        return response

    def optimize_waypoints(self,
                           waypoints: List[Tuple[float, float]],
                           start_index: int = 0,
                           end_index: int = None) -> Dict:
        """
        Optimize the order of waypoints for shortest travel time
        
        Args:
            waypoints: List of waypoints to optimize
            start_index: Index of starting waypoint
            end_index: Index of ending waypoint (None for round trip)
            
        Returns:
            Optimized route results
        """
        # Format waypoints for optimization
        query_points = ":".join([f"{lat},{lon}" for lat, lon in waypoints])

        params = {
            'query': query_points,
            'computeOptimalOrder': 'true',
            'travelMode': 'car'
        }

        if start_index is not None:
            params['optimizeWaypointOrder'] = f"{start_index}"

        if end_index is not None:
            params['optimizeWaypointOrder'] = f"{start_index}:{end_index}"

        response = self.client._make_request('route/directions/json', params)
        return response

## Examples

In [8]:
routing = RoutingDemo(azure_maps)

### Example 1: Basic Route Calculation

In [9]:
# Define waypoints
nyc_coords = (40.7128, -74.0060)  # New York City
philly_coords = (39.9526, -75.1652)  # Philadelphia

route_result = routing.calculate_route([nyc_coords, philly_coords])

if route_result and 'routes' in route_result:
    route = route_result['routes'][0]
    summary = route['summary']

    print(f"Route Summary:")
    print(f"  Distance: {summary['lengthInMeters'] / 1000:.1f} km")
    print(f"  Travel Time: {summary['travelTimeInSeconds'] // 60} minutes")
    print(
        f"  Traffic Delay: {summary.get('trafficDelayInSeconds', 0) // 60} minutes"
    )
    print(f"  Departure Time: {summary.get('departureTime', 'N/A')}")
    print(f"  Arrival Time: {summary.get('arrivalTime', 'N/A')}")

Route Summary:
  Distance: 152.1 km
  Travel Time: 94 minutes
  Traffic Delay: 0 minutes
  Departure Time: 2025-09-01T04:02:05-04:00
  Arrival Time: 2025-09-01T05:36:58-04:00


### Example 2: Multi-stop Route

In [10]:
# Define multiple waypoints
east_coast_trip = [
    (40.7128, -74.0060),  # New York City
    (39.9526, -75.1652),  # Philadelphia
    (38.9072, -77.0369),  # Washington DC
    (35.2271, -80.8431),  # Charlotte
    (33.7490, -84.3880),  # Atlanta
]

city_names = [
    "New York", "Philadelphia", "Washington DC", "Charlotte", "Atlanta"
]

multi_route = routing.calculate_route(east_coast_trip, route_type="fastest")

if multi_route and 'routes' in multi_route:
    route = multi_route['routes'][0]
    summary = route['summary']

    print(f"Multi-stop Route Summary:")
    print(f"  Total Distance: {summary['lengthInMeters'] / 1000:.1f} km")
    print(
        f"  Total Travel Time: {summary['travelTimeInSeconds'] // 3600:.1f} hours"
    )

    # Show leg details
    if 'legs' in route:
        print(f"\nLeg Details:")
        for i, leg in enumerate(route['legs']):
            leg_summary = leg['summary']
            start_city = city_names[i]
            end_city = city_names[i + 1]

            print(f"  {start_city} → {end_city}:")
            print(
                f"    Distance: {leg_summary['lengthInMeters'] / 1000:.1f} km")
            print(
                f"    Time: {leg_summary['travelTimeInSeconds'] // 60} minutes"
            )

Multi-stop Route Summary:
  Total Distance: 1409.1 km
  Total Travel Time: 14.0 hours

Leg Details:
  New York → Philadelphia:
    Distance: 152.1 km
    Time: 94 minutes
  Philadelphia → Washington DC:
    Distance: 221.7 km
    Time: 168 minutes
  Washington DC → Charlotte:
    Distance: 642.3 km
    Time: 354 minutes
  Charlotte → Atlanta:
    Distance: 393.0 km
    Time: 237 minutes


### Example 3: Different Transportation Modes

In [11]:
# Compare different modes between two points
start_point = (40.7589, -73.9851)  # Times Square
end_point = (40.7505, -73.9934)  # Empire State Building

transportation_modes = ["pedestrian", "bicycle", "car", "taxi"]

mode_results = {}
for mode in transportation_modes:
    result = routing.calculate_route([start_point, end_point],
                                     travel_mode=mode)

    if result and 'routes' in result:
        summary = result['routes'][0]['summary']
        mode_results[mode] = {
            'distance': summary['lengthInMeters'] / 1000,
            'time': summary['travelTimeInSeconds'] / 60
        }

print(
    "Transportation Mode Comparison (Times Square to Empire State Building):")
for mode, data in mode_results.items():
    print(
        f"  {mode.title()}: {data['distance']:.2f} km, {data['time']:.1f} minutes"
    )

Transportation Mode Comparison (Times Square to Empire State Building):
  Pedestrian: 1.27 km, 15.2 minutes
  Bicycle: 1.28 km, 5.2 minutes
  Car: 1.53 km, 7.0 minutes
  Taxi: 1.53 km, 7.1 minutes


### Example 4: Route with Avoidances

In [12]:
# Calculate route avoiding tolls and motorways
route_normal = routing.calculate_route([nyc_coords, philly_coords])
route_no_tolls = routing.calculate_route([nyc_coords, philly_coords],
                                         avoid=["tollRoads"])
route_no_highways = routing.calculate_route([nyc_coords, philly_coords],
                                            avoid=["motorways"])

routes_comparison = {
    "Normal Route": route_normal,
    "No Tolls": route_no_tolls,
    "No Highways": route_no_highways
}

print("Route Avoidance Comparison (NYC to Philadelphia):")
for route_name, result in routes_comparison.items():
    if result and 'routes' in result:
        summary = result['routes'][0]['summary']
        distance = summary['lengthInMeters'] / 1000
        time = summary['travelTimeInSeconds'] / 60
        print(f"  {route_name}: {distance:.1f} km, {time:.0f} minutes")

Route Avoidance Comparison (NYC to Philadelphia):
  Normal Route: 152.1 km, 95 minutes
  No Tolls: 150.8 km, 112 minutes
  No Highways: 160.5 km, 174 minutes


### Example 5: Create Route Visualization Map

In [13]:
def create_route_map(route_data: Dict,
                     waypoints: List[Tuple[float, float]],
                     waypoint_names: List[str] = None) -> folium.Map:
    """
    Create folium map with route visualization
    
    Args:
        route_data: Route calculation result
        waypoints: List of waypoint coordinates
        waypoint_names: Optional names for waypoints
        
    Returns:
        Folium map with route
    """
    if not route_data or 'routes' not in route_data:
        return None

    route = route_data['routes'][0]

    # Calculate map center
    center_lat = sum(lat for lat, lon in waypoints) / len(waypoints)
    center_lon = sum(lon for lat, lon in waypoints) / len(waypoints)

    # Create map
    m = folium.Map(location=[center_lat, center_lon],
                   zoom_start=8,
                   tiles='OpenStreetMap')

    # Add route polyline if available
    if 'legs' in route:
        route_coords = []
        for leg in route['legs']:
            if 'points' in leg:
                for point in leg['points']:
                    if 'latitude' in point and 'longitude' in point:
                        route_coords.append(
                            [point['latitude'], point['longitude']])

        if route_coords:
            folium.PolyLine(locations=route_coords,
                            color='blue',
                            weight=4,
                            opacity=0.8).add_to(m)

    # Add waypoint markers
    for i, (lat, lon) in enumerate(waypoints):
        name = waypoint_names[i] if waypoint_names and i < len(
            waypoint_names) else f"Point {i+1}"

        icon_color = 'green' if i == 0 else 'red' if i == len(
            waypoints) - 1 else 'blue'
        icon_symbol = 'play' if i == 0 else 'stop' if i == len(
            waypoints) - 1 else 'pause'

        folium.Marker(location=[lat, lon],
                      popup=f"<b>{name}</b>",
                      tooltip=name,
                      icon=folium.Icon(color=icon_color,
                                       icon=icon_symbol)).add_to(m)

    return m


# Create route map for the multi-stop trip
if multi_route:
    route_map = create_route_map(multi_route, east_coast_trip, city_names)
    if route_map:

        output_file = os.path.join(RESULTS_DIR, "azure_maps_route_demo.html")
        route_map.save(output_file)
        print(f"✅ Map saved as {output_file}")

✅ Map saved as results/azure_maps_route_demo.html


In [14]:
IFrame(src=output_file, width=860, height=640)

In [15]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/azure_maps_route_demo.html

### Example 6: Route Optimization

In [16]:
# Define delivery points around a city
seattle_center = (47.6062, -122.3321)

delivery_points = [
    (47.6062, -122.3321),  # Seattle Center (start)
    (47.6205, -122.3493),  # Capitol Hill
    (47.6097, -122.3331),  # Downtown
    (47.6587, -122.3198),  # University District
    (47.5952, -122.3316),  # Georgetown
    (47.6694, -122.3843),  # Ballard
    (47.6062, -122.3321),  # Return to start
]

location_names = [
    "Start/End", "Capitol Hill", "Downtown", "University District",
    "Georgetown", "Ballard", "Start/End"
]

# Calculate optimized route
optimized_route = routing.optimize_waypoints(
    delivery_points[:-1])  # Exclude duplicate end point

if optimized_route and 'routes' in optimized_route:
    route = optimized_route['routes'][0]
    summary = route['summary']

    print("Optimized Delivery Route:")
    print(f"  Total Distance: {summary['lengthInMeters'] / 1000:.1f} km")
    print(f"  Total Time: {summary['travelTimeInSeconds'] / 60:.0f} minutes")

    # Show optimized order if available
    if 'optimizedWaypoints' in route:
        print("\nOptimized waypoint order:")
        for i, waypoint_index in enumerate(route['optimizedWaypoints']):
            if waypoint_index < len(location_names):
                print(f"  {i+1}. {location_names[waypoint_index]}")

Optimized Delivery Route:
  Total Distance: 32.2 km
  Total Time: 58 minutes
